### Perguntas dessa tarefa:

1. Inicialmente iremos preparar o ambiente, definindo o diretório onde nosso código será desenvolvido. Para este diretório iremos copiar o arquivo nomes_aleatorios.txt.

   Após, em nosso script Python, devemos importar as bibliotecas necessárias:
   ```python
   from pyspark.sql import SparkSession
   from pyspark import SparkContext, SQLContext
   ```
   Aplicando as bibliotecas do Spark, podemos definir a Spark Session e sobre ela definir o Context para habilitar o módulo SQL
   ```python
   spark = SparkSession \
                   .builder \
                   .master("local[*]")\
                   .appName("Exercicio Intro") \
                   .getOrCreate()
   ```
   Nesta etapa, adicione código para ler o arquivo nomes_aleatorios.txt através do comando `spark.read.csv`. Carregue-o para dentro de um dataframe chamado `df_nomes` e, por fim, liste algumas linhas através do método `show`. Exemplo: `df_nomes.show(5)`


In [82]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, when, col, lit
from pyspark.sql.types import StringType
import random

# Inicializando a Spark Session
spark = SparkSession \
    .builder \
    .appName("Exercicio Intro") \
    .getOrCreate()

# 1. Lendo o arquivo nomes_aleatorios.txt
df_nomes = spark.read.csv("nomes_aleatorios.txt", header=False, inferSchema=True)
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



2. No Python, é possível acessar uma coluna de um objeto dataframe pelo atributo (por exemplo `df_nomes.nome`) ou por índice (`df_nomes['nome']`). Enquanto a primeira forma é conveniente para a exploração de dados interativos, você deve usar o formato de índice, pois caso algum nome de coluna não esteja de acordo seu código irá falhar.

   Como não informamos no momento da leitura do arquivo, o Spark não identificou o Schema por padrão e definiu todas as colunas como string. Para ver o Schema, use o método `df_nomes.printSchema()`.

   Nesta etapa, será necessário adicionar código para renomear a coluna para Nomes, imprimir o esquema e mostrar 10 linhas do dataframe.

In [83]:
# 2. Renomeando a coluna para 'Nomes'
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.show(10)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



3. Ao dataframe (`df_nomes`), adicione nova coluna chamada `Escolaridade` e atribua para cada linha um dos três valores de forma aleatória: Fundamental, Médio ou Superior.
   Para esta etapa, evite usar funções de iteração, como por exemplo: `for`, `while`, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

In [84]:
# 3. Adicionando a nova coluna 'Escolaridade' com valores aleatórios
df_nomes = df_nomes.withColumn("Escolaridade", when(rand() < 0.33, "Fundamental").
                                              when(rand() < 0.66, "Médio").
                                              otherwise("Superior"))

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|    Superior|
|    Jamie Russell|       Médio|
|   Edward Kistler|    Superior|
|    Sheila Maurer| Fundamental|
| Donald Golightly|       Médio|
|       David Gray| Fundamental|
|      Joy Bennett|       Médio|
|      Paul Kriese|    Superior|
|Berniece Ornellas|       Médio|
|    Brian Farrell| Fundamental|
+-----------------+------------+
only showing top 10 rows



4. Ao dataframe (`df_nomes`), adicione nova coluna chamada `Pais` e atribua para cada linha o nome de um dos 13 países da América do Sul, de forma aleatória.
   Para esta etapa, evite usar funções de iteração, como por exemplo: `for`, `while`, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

In [85]:
# 4. Definindo a lista de países da América do Sul 
paises_americadosul = ['Argentina', 'Bolívia', 'Brasil', 'Chile', 'Colômbia', 'Equador', 'Guiana', 'Paraguai', 'Peru', 'Suriname', 'Uruguai', 'Venezuela']
escolaridades = ['Fundamental', 'Médio', 'Superior']

# Função UDF para selecionar um país aleatório
def random_country():
    return random.choice(paises_americadosul)

# Registrando a função UDF
spark.udf.register("random_country", random_country, StringType())

# Adicionando a nova coluna 'Pais' com valores aleatórios
df_nomes = df_nomes.withColumn("Pais", expr("random_country()"))

# Exibindo o DataFrame resultante
df_nomes.show(10)

+-----------------+------------+---------+
|            Nomes|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet|    Superior|   Brasil|
|    Jamie Russell|       Médio|   Brasil|
|   Edward Kistler|    Superior|Venezuela|
|    Sheila Maurer| Fundamental| Paraguai|
| Donald Golightly|       Médio|  Bolívia|
|       David Gray| Fundamental|  Bolívia|
|      Joy Bennett|       Médio| Colômbia|
|      Paul Kriese|    Superior|    Chile|
|Berniece Ornellas|       Médio|     Peru|
|    Brian Farrell| Fundamental|   Brasil|
+-----------------+------------+---------+
only showing top 10 rows



5. Ao dataframe (`df_nomes`), adicione nova coluna chamada `AnoNascimento` e atribua para cada linha um valor de ano entre 1945 e 2010, de forma aleatória.
   Para esta etapa, evite usar funções de iteração, como por exemplo: `for`, `while`, entre outras. Dê preferência aos métodos oferecidos para próprio Spark.

In [86]:
# 5. Adicionando a coluna 'AnoNascimento' com valores aleatórios entre 1945 e 2010
df_nomes = df_nomes.withColumn("AnoNascimento", (lit(1945) + (rand(seed=42) * (2010 - 1945 + 1)).cast("int")))
df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet|    Superior|Venezuela|         1985|
|    Jamie Russell|       Médio|  Uruguai|         1978|
|   Edward Kistler|    Superior|   Guiana|         1999|
|    Sheila Maurer| Fundamental| Paraguai|         1962|
| Donald Golightly|       Médio|     Peru|         1989|
|       David Gray| Fundamental|  Bolívia|         1979|
|      Joy Bennett|       Médio| Suriname|         2010|
|      Paul Kriese|    Superior|    Chile|         1949|
|Berniece Ornellas|       Médio|Argentina|         2008|
|    Brian Farrell| Fundamental| Colômbia|         1997|
+-----------------+------------+---------+-------------+
only showing top 10 rows



6. Usando o método `select` do dataframe (`df_nomes`), selecione as pessoas que nasceram neste século. Armazene o resultado em outro dataframe chamado `df_select` e mostre 10 nomes deste.

In [87]:
# 6. Selecionando as pessoas que nasceram neste século
df_select = df_nomes.filter(df_nomes["AnoNascimento"] >= 2000)
print("Pessoas que nasceram neste século:")
df_select.select("Nomes", "AnoNascimento").show(10)

Pessoas que nasceram neste século:
+-----------------+-------------+
|            Nomes|AnoNascimento|
+-----------------+-------------+
|      Joy Bennett|         2010|
|Berniece Ornellas|         2008|
|      Albert Leef|         2000|
|     Rebecca Snow|         2003|
|  Kenneth Rayburn|         2001|
|    Milton Dillon|         2002|
|       Ned Tester|         2010|
|    Lynne Dustman|         2003|
|    George Miller|         2002|
| Cristina Sheston|         2006|
+-----------------+-------------+
only showing top 10 rows



7. Usando Spark SQL repita o processo da Pergunta 6. Lembre-se que, para trabalharmos com SparkSQL, precisamos registrar uma tabela temporária e depois executar o comando SQL. Abaixo um exemplo de como executar comandos SQL com SparkSQL:

   ```python
   df_nomes.createOrReplaceTempView("pessoas")
   spark.sql("select * from pessoas").show()
   ```

In [88]:
# 7. Usando Spark SQL para selecionar as pessoas que nasceram neste século
df_nomes.createOrReplaceTempView("pessoas")
df_select_sql = spark.sql("SELECT Nomes, AnoNascimento FROM pessoas WHERE AnoNascimento >= 2000")
print("Pessoas que nasceram neste século (usando Spark SQL):")
df_select_sql.show(10)

Pessoas que nasceram neste século (usando Spark SQL):
+-----------------+-------------+
|            Nomes|AnoNascimento|
+-----------------+-------------+
|      Joy Bennett|         2010|
|Berniece Ornellas|         2008|
|      Albert Leef|         2000|
|     Rebecca Snow|         2003|
|  Kenneth Rayburn|         2001|
|    Milton Dillon|         2002|
|       Ned Tester|         2010|
|    Lynne Dustman|         2003|
|    George Miller|         2002|
| Cristina Sheston|         2006|
+-----------------+-------------+
only showing top 10 rows



8. Usando o método `select` do Dataframe `df_nomes`, Conte o número de pessoas que são da geração Millennials (nascidos entre 1980 e 1994) no Dataset

In [89]:
# 8. Contando o número de pessoas que são da geração Millennials (nascidos entre 1980 e 1994)
count_millennials = df_nomes.filter((df_nomes["AnoNascimento"] >= 1980) & (df_nomes["AnoNascimento"] <= 1994)).count()
print("Número de pessoas da geração Millennials:", count_millennials)

Número de pessoas da geração Millennials: 2272333


9. Repita o processo da Pergunta 8 utilizando Spark SQL

In [90]:
# 9. Repetindo o processo da Pergunta 8 utilizando Spark SQL
count_millennials_sql = spark.sql("SELECT COUNT(*) FROM pessoas WHERE AnoNascimento >= 1980 AND AnoNascimento <= 1994").collect()[0][0]
print("Número de pessoas da geração Millennials (usando Spark SQL):", count_millennials_sql)

Número de pessoas da geração Millennials (usando Spark SQL): 2272333


10. Usando Spark SQL, obtenha a quantidade de pessoas de cada país para uma das gerações abaixo. Armazene o resultado em um novo dataframe e depois mostre todas as linhas em ordem crescente de Pais, Geração e Quantidade

   - Baby Boomers – nascidos entre 1944 e 1964;
   - Geração X – nascidos entre 1965 e 1979;
   - Millennials (Geração Y) – nascidos entre 1980 e 1994;
   - Geração Z – nascidos entre 1995 e 2015.

In [91]:
# 10. Obtendo a quantidade de pessoas de cada país para a geração Millennials
df_millennials = df_nomes.filter((df_nomes["AnoNascimento"] >= 1980) & (df_nomes["AnoNascimento"] <= 1994))
df_millennials_grouped = df_millennials.groupBy("Pais").count().orderBy("Pais")
print("Quantidade de pessoas de cada país para a geração Millennials:")
df_millennials_grouped.show(df_millennials_grouped.count())

Quantidade de pessoas de cada país para a geração Millennials:
+---------+------+
|     Pais| count|
+---------+------+
|Argentina|189584|
|  Bolívia|189326|
|   Brasil|189387|
|    Chile|189175|
| Colômbia|189207|
|  Equador|189140|
|   Guiana|188921|
| Paraguai|189481|
|     Peru|189258|
| Suriname|189567|
|  Uruguai|189439|
|Venezuela|189848|
+---------+------+

